In [1]:
import pandas as pd
import requests 
from bs4 import BeautifulSoup as BS
from IPython.core.display import HTML
import json
import re

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_deadly_earthquakes_since_1900'

In [3]:
response = requests.get(url)

In [4]:
wiki_page = BS(response.text)

In [5]:
wiki_table = wiki_page.find('table',{'class':'wikitable'})

In [6]:
wiki_list = pd.read_html(str(wiki_table))

In [7]:
wiki_df = pd.DataFrame(wiki_list[0])

In [8]:
#regex to get footnoes out of other deaths column 
#point1 = wiki_df.iloc[1338,-1]

In [9]:
#regex to get footnoes out of other deaths column 
#re.match('(\d+)', point1)

In [10]:
wiki_df['Other Source Deaths'].unique()

array([nan, '3500', '8000+', '46', '164[6]', '1500[6]', '105000', '1404',
       '1', '120', '200', '380', '2041', '33[8]', '45000[9]', '2489[10]',
       '10', '26271[11] 26000[12]', '68',
       '231000*[13] 283000*[14] 227898*[15]', '41', '60[16]', '215', '34',
       '295', '79', '189', '1115', '222,517[17]', '521[19]', '42', '2698',
       '185', '15894[20]', '150+', '111+', '601 (as of October 30, 2011)',
       '4340+', '118', '105'], dtype=object)

In [11]:
#doing this to get the index value of the point in question 
wiki_df[wiki_df['Other Source Deaths'] == '231000*[13] 283000*[14] 227898*[15]']


,Origin (UTC),Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths
1238,2004-12-26 00:58,Indonesia (see 2004 Indian Ocean earthquake),3.287,95.972,30,9.1 Mw,"T,L",NaN,297200.0,NaN,NaN,231000*[13] 283000*[14] 227898*[15]


In [12]:
wiki_df[wiki_df['Other Source Deaths'] == '222,517[17]']

,Origin (UTC),Present-day country and link to Wikipedia article,Lat,Long,Depth (km),Magnitude,Secondary Effects,PDE Shaking Deaths,PDE Total Deaths,Utsu Total Deaths,EM-DAT Total Deaths,Other Source Deaths
1328,2010-01-12 17:00,Haiti (see 2010 Haiti earthquake),18.457,-72.533,13,7.0 Mw,NaN,NaN,NaN,NaN,NaN,"222,517[17]"


In [13]:
wiki_df.iloc[1237,-1] = '283000'
wiki_df.iloc[1327,-1] = '222517'

In [14]:
wiki_df['Other Source Deaths'] = wiki_df['Other Source Deaths'].str.extract('(\d+)')
wiki_df['Other Source Deaths'] = wiki_df['Other Source Deaths'].fillna(0)

In [15]:
wiki_df['Other Source Deaths'] = wiki_df['Other Source Deaths'].astype(int)

In [16]:
point2 = wiki_df.iloc[1335,5]

In [17]:
re.match('\d{1}\.\d{1}', point2)

<re.Match object; span=(0, 3), match='9.0'>

In [18]:
wiki_df['Magnitude'] = wiki_df['Magnitude'].str.extract('(\d{1}\.\d{1})')

In [19]:
# also go this wrong wiki_df.fillna(0)
wiki_df['Magnitude'] = wiki_df['Magnitude'].astype(float)

In [20]:
wiki_df['Deaths'] = wiki_df[['PDE Total Deaths','Utsu Total Deaths','EM-DAT Total Deaths','Other Source Deaths']].max(axis=1)

In [21]:
wiki_df['Present-day country and link to Wikipedia article'] = wiki_df['Present-day country and link to Wikipedia article'].str.replace('(\s\(.*)','')

In [22]:
wiki_df.query('Magnitude > 5.0 or (Magnitude < 5.0 and `Secondary Effects` == "L" and Deaths > 10)', inplace = True)

In [23]:
wiki_df.to_csv('cleandata.csv', index=False)